In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from bs4 import BeautifulSoup
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/Project/BBC News Train.csv'
df = pd.read_csv(file_path)

In [ ]:
df

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [ ]:
df['Category'].value_counts()

,count
Category,
sport,346
business,336
politics,274
entertainment,273
tech,261


In [ ]:

file_path = '/content/drive/MyDrive/Project/News_Category_Dataset_v3.json'
# Try reading the file with the 'lines' parameter set to True
try:
    df0 = pd.read_json(file_path, lines=True)
except ValueError as e:
    # If the above fails, try reading it as a list of JSON objects
    import json
    with open(file_path, 'r') as f:
        data = '[' + f.read().replace('}\n{', '}, {') + ']'  # Assuming each line is a JSON object

    df0 = pd.read_json(data)

In [ ]:
df0

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [ ]:
df0.drop(columns=['link','authors','date','headline'],inplace=True)

In [ ]:
df0

,category,short_description
0,U.S. NEWS,Health experts said it is too early to predict...
1,U.S. NEWS,He was subdued by passengers and crew when he ...
2,COMEDY,"""Until you have a dog you don't understand wha..."
3,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
...,...,...
209522,TECH,Verizon Wireless and AT&T are already promotin...
209523,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
209524,SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
209525,SPORTS,CORRECTION: An earlier version of this story i...


In [ ]:
df0['category'].value_counts()

,count
category,
POLITICS,35602
WELLNESS,17945
ENTERTAINMENT,17362
TRAVEL,9900
STYLE & BEAUTY,9814
PARENTING,8791
HEALTHY LIVING,6694
QUEER VOICES,6347
FOOD & DRINK,6340


In [ ]:
df0.sample(10)

,category,short_description
124789,WOMEN,"""If your 'friend' expresses interest in anothe..."
64755,ARTS & CULTURE,Sean Aaron Carmon went from wanting to go it a...
162690,WELLNESS,Every challenge in our lives presents us with ...
183042,WELLNESS,"Tell us how it goes in the comments below, and..."
160351,CRIME,“I wasn’t just going to let someone leave that...
128711,WOMEN,It was an auspicious start for women in techno...
147877,QUEER VOICES,"Abercrombie, who spearheaded the marriage equa..."
62502,HEALTHY LIVING,It is COMMON to leak urine when you are coughi...
20442,POLITICS,Are Republicans shooting themselves in the foo...
95126,ENTERTAINMENT,


In [ ]:
df1=df0[(df0['category']=='BUSINESS')|(df0['category']=='SPORTS')|(df0['category']=='TECH')|(df0['category']=='ENTERTAINMENT')|(df0['category']=='POLITICS')|(df0['category']=='SCIENCE')]

In [ ]:
df1['category'].value_counts()

,count
category,
POLITICS,35602
ENTERTAINMENT,17362
BUSINESS,5992
SPORTS,5077
SCIENCE,2206
TECH,2104


In [ ]:
def cleanText(text):
      text = BeautifulSoup(text, "lxml").text
      text = re.sub(r'\|\|\|', r' ', text)
      text = re.sub(r'http\S+', r'<URL>', text)
      text = re.sub(r'[http://]?www.\w+\.\w+','',text)
      text = re.sub(r'@\w+', '', text)
      text = re.sub(r'\d+', '', text)
      text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
      return text
df['Text'] = df['Text'].apply(cleanText)
df1['short_description'] = df1['short_description'].apply(cleanText)

<ipython-input-14-fadb72d29b67>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text
<ipython-input-14-fadb72d29b67>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "lxml").text
<ipython-input-14-fadb72d29b67>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['short_description'] = df1['short_description'].apply(cleanText)


In [ ]:
df


,ArticleId,Text,Category
0,1833,worldcom exboss launches defence lawyers defen...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in m payout eighteen former enron...,business
...,...,...,...
1485,857,double eviction from big brother model caprice...,entertainment
1486,325,dj double act revamp chart show dj duo jk and ...,entertainment
1487,1590,weak dollar hits reuters revenues at media gro...,business
1488,1587,apple ipod family expands market apple has exp...,tech


In [ ]:
df1

,category,short_description
13,TECH,One mans claims that he scammed people on the ...
17,SPORTS,Maury Wills who helped the Los Angeles Dodgers...
20,ENTERTAINMENT,For the past months Hollywood has effectively...
21,POLITICS,President issues vow as tensions with China rise
24,POLITICS,An annual celebration took on a different feel...
...,...,...
209522,TECH,Verizon Wireless and ATT are already promoting...
209523,SPORTS,Afterward Azarenka more effusive with the pres...
209524,SPORTS,Leading up to Super Bowl XLVI the most talked ...
209525,SPORTS,CORRECTION An earlier version of this story in...


In [ ]:
def load_imdb_data(df):
    texts = df['Text'].tolist()
    labels = [0 if label == "sport"
              else 1 if label == "business"
              else 2 if label == "politics"
              else 3 if label == "entertainment"
              else 4
              for label in df['Category'].tolist()]
    return texts, labels

In [ ]:
texts, labels = load_imdb_data(df)

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            x = self.dropout(pooled_output)
            logits = self.fc(x)
            return logits

In [ ]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad() # at each batch you have to reset the grads to 0 so that you restrart calculating the grads
        input_ids = batch['input_ids'].to(device) # moves the data to the device ( cpu or gpu )
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward() # This line computes the gradients of the loss with respect to the model parameters. This is necessary for updating the model weights during training.
        optimizer.step() # This line updates the model parameters using the optimizer, applying the computed gradients to minimize the loss.
        scheduler.step() # This line updates the learning rate according to the specified schedule.

In [ ]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad(): # Disable Gradient Tracking
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [ ]:
def predict_category(text, model, tokenizer, device, max_length=256):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
    return  ("sport"  if preds.item() == 0 else "business"
                            if preds.item() == 1 else "politics"
                            if preds.item() == 2 else "entertainment"
                            if preds.item() == 3 else "Tech")


In [ ]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 5
max_length = 256
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
# It creates a scheduler that modifies the learning rate according to a linear schedule after a warmup period.
# In this case, it is set to 0, meaning there is no warmup phase— the learning rate will start at its initial value from the beginning.

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

Epoch 1/4
Validation Accuracy: 0.9765
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        63
           1       0.96      0.97      0.97        75
           2       0.93      0.98      0.96        56
           3       1.00      1.00      1.00        46
           4       1.00      0.93      0.96        58

    accuracy                           0.98       298
   macro avg       0.98      0.98      0.98       298
weighted avg       0.98      0.98      0.98       298

Epoch 2/4
Validation Accuracy: 0.9698
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        63
           1       0.94      0.99      0.96        75
           2       0.96      0.98      0.97        56
           3       0.98      1.00      0.99        46
           4       1.00      0.88      0.94        58

    accuracy                           0.97       298
   macro avg       0.97      0.97      0.97       298
w

In [30]:
import joblib

In [31]:
joblib.dump(model, 'BERT.joblib')

['BERT.joblib']